In [ ]:
# !pip install dill
# !pip install lime

In [3]:
import pandas as pd
import numpy as np
# import joblib
# import dill
# from lime.lime_tabular import LimeTabularExplainer
from google.colab import userdata
%run userdata.get('gral_funcs_box_data')
%run userdata.get('funcs_model_to_short')
%run userdata.get('cleaner_raw_data')
%run userdata.get('clases_predictor')
%xmode #verbose
import warnings
warnings.filterwarnings('ignore')
cluster = joblib.load(userdata.get('cluster'))
scaler = joblib.load(userdata.get('scaler'))
model = joblib.load(userdata.get('model'))
with open(userdata.get('explainer'), 'rb') as file:
  explainer = dill.load(file)
pd.set_option('display.max_columns', 50)
# lectura de la data nueva
bo = pd.read_csv(userdata.get('boxers_info'))
fi = pd.read_csv(userdata.get('fight_info_real'))
fr = pd.read_csv(userdata.get('fight_results_real'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=ac27e67900a62d55a401e29f652a064e93de2e059d33a808cbc2277a3985f774
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/basic.py:375: UserWarning: Error changing user exception modes.
Unrecognized mode in FormattedTB: <#verbose>
Valid modes: ['Plain', 'Context', 'Verbose', 'Minimal']
  warn('Error changing %s exception modes.\n%s' %


In [ ]:
# merge and split new data
ndf, new_id_df, new_other_cols = box_data_merger_spliter(bo,fi,fr)

# separate ndf to get the columns with res nan values
data = ndf[ndf.result.isna()]

In [ ]:
data

,fight_id,result,endround,endtype,boxer1,boxer2,birth_b1,height_b1,b1_w,b1_wk,b1_d,b1_l,b1_lk,b1_bet,len_text_boxer1,birth_b2,height_b2,b2_w,b2_wk,b2_d,b2_l,b2_lk,b2_bet,len_text_boxer2
35,36.0,NaN,NaN,NaN,jesse rodriguez,sunny edwards,2000,163,18.0,11.0,0.0,0.0,0.0,1.080,6005,1996,160,20.0,4.0,0.0,0.0,0.0,1.140,7051
61,64.0,NaN,NaN,NaN,yoanki urrutia,fiodor czerkaszyn,1996,188,13.0,5.0,0.0,0.0,0.0,3.100,0,1990,170,22.0,14.0,0.0,1.0,0.0,1.363,0
62,65.0,NaN,NaN,NaN,eduardo nunez,oscar escandon,1997,168,24.0,24.0,0.0,1.0,0.0,1.100,13683,1980,173,27.0,18.0,0.0,6.0,4.0,7.000,2008
65,68.0,NaN,NaN,NaN,joe cordina,edward vazquez,1992,175,16.0,9.0,0.0,0.0,0.0,1.083,6351,1995,170,15.0,3.0,0.0,1.0,0.0,8.000,0
66,69.0,NaN,NaN,NaN,robeisy ramirez,rafael espinoza,1994,168,13.0,8.0,0.0,1.0,0.0,1.071,7087,1994,185,21.0,18.0,0.0,0.0,0.0,8.000,0
67,70.0,NaN,NaN,NaN,emanuel navarrete,robson conceicao,1994,180,38.0,31.0,0.0,1.0,0.0,1.100,12665,1996,170,17.0,8.0,0.0,2.0,0.0,7.000,7632
68,71.0,NaN,NaN,NaN,shakur stevenson,edwin de los santos,1997,175,20.0,10.0,0.0,0.0,0.0,1.083,9892,1999,173,16.0,14.0,0.0,1.0,0.0,8.000,0
69,72.0,NaN,NaN,NaN,diego pacheco,marcelo esteban coceres,2001,193,19.0,16.0,0.0,0.0,0.0,1.050,1466,1983,175,32.0,18.0,1.0,5.0,1.0,10.000,0
70,73.0,NaN,NaN,NaN,david benavidez,demetrius andrade,1996,188,27.0,23.0,0.0,0.0,0.0,1.300,12090,1988,178,32.0,19.0,0.0,0.0,0.0,3.500,24084
72,75.0,NaN,NaN,NaN,michael conlan,jordan gill,1992,173,18.0,9.0,0.0,2.0,2.0,1.167,9592,1994,170,27.0,8.0,1.0,2.0,2.0,5.500,3207


### making predictions

In [ ]:
predictor = Model_predictions(cluster, scaler, model, explainer)
preds = predictor.make_prediction(data, new_id_df)

In [ ]:
preds.head(3)

,initial_index,fight_id,result,endround,endtype,cluster,boxer1_pred,goodpred,prob_win,data_cuant,confidence,boxer1,boxer2,dif_bet,dif_len_text,dif_edad,dif_height,dif_w,dif_wk_perc,dif_l,dif_lk_perc,modelo,b1_id,b2_id,explanation,date_model_pred
0,35,36.0,NaN,NaN,NaN,0,0,,0.475330,"fail 19 % of 1150 (val, test)","WARNING: .45 to .65, fail 50 %",jesse rodriguez,sunny edwards,1.08 - 1.14 = -0.06,6005 - 7051 = -1046,2000 - 1996 = 4,163 - 160 = 3,18.0 - 20.0 = -2.0,0.61 - 0.2 = 0.41,0.0 - 0.0 = 0.0,0.0 - 0.0 = 0.0,SVC(probability=True),6502,6532,"{'len_text_boxer2 > -0.14': -0.2311, 'birth_b1...",2023-11-13 00:48:25.158735
1,61,64.0,NaN,NaN,NaN,0,1,,0.935191,"fail 19 % of 1150 (val, test)",,yoanki urrutia,fiodor czerkaszyn,3.1 - 1.36 = 1.74,0 - 0 = 0,1996 - 1990 = 6,188 - 170 = 18,13.0 - 22.0 = -9.0,0.38 - 0.64 = -0.25,0.0 - 1.0 = -1.0,0.0 - 0.0 = 0.0,SVC(probability=True),6553,6363,"{'birth_b1 > 0.79': 0.1979, 'b1_wk <= -0.66': ...",2023-11-13 00:48:25.158735
2,62,65.0,NaN,NaN,NaN,0,1,,0.935475,"fail 19 % of 1150 (val, test)",,eduardo nunez,oscar escandon,1.1 - 7.0 = -5.9,13683 - 2008 = 11675,1997 - 1980 = 17,168 - 173 = -5,24.0 - 27.0 = -3.0,1.0 - 0.67 = 0.33,1.0 - 6.0 = -5.0,0.0 - 0.67 = -0.67,SVC(probability=True),6554,2775,"{'birth_b1 > 0.79': 0.1987, 'len_text_boxer1 >...",2023-11-13 00:48:25.158735


### logging data

In [ ]:
logger = Predictions_logger(userdata.get('mades_predictions'))

In [ ]:
logger.record_data(preds)

{'cluster': 0, 'boxer1_pred': 0, 'prob_win': 0.475, 'data_cuant': 'fail 19 % of 1150 (val, test)', 'confidence': 'WARNING: .45 to .65, fail 50 %'}
{'boxer1': 'jesse rodriguez', 'boxer2': 'sunny edwards', 'dif_bet': '1.08 - 1.14 = -0.06'}
{'dif_len_text': '6005 - 7051 = -1046', 'dif_edad': '2000 - 1996 = 4', 'dif_height': '163 - 160 = 3'}
{'dif_w': '18.0 - 20.0 = -2.0', 'dif_wk_perc': '0.61 - 0.2 = 0.41', 'dif_l': '0.0 - 0.0 = 0.0', 'dif_lk_perc': '0.0 - 0.0 = 0.0'}
{'modelo': 'SVC(probability=True)', 'explanation': {'len_text_boxer2 > -0.14': -0.2311, 'birth_b1 > 0.79': 0.1981, 'b1_wk <= -0.66': -0.1542, 'cluster <= 0.12': -0.0884, 'b1_more_fame <= -1.62': -0.0658, '-1.02 < b1_momios_menores <= 0.98': 0.0651, 'height_b2 <= -0.51': 0.055, '-0.81 < b1_lk <= -0.38': 0.0534, 'birth_b2 > 0.63': -0.0496, '-0.73 < b2_lk <= -0.26': -0.0495}}

Opinion personal (1 - boxer1 win, 0 - boxer1 loss, c - close): cualquiera podria ganar - aunque tiene mejor ko perc el boxeador 1 y tiene los momios mas 

In [ ]:
preds2 = logger.read_file()

In [ ]:
preds2

,fight_id,self_pred,cluster,boxer1_pred,prob_win,data_cuant,confidence,boxer1,boxer2,dif_bet,dif_len_text,dif_edad,dif_height,dif_w,dif_wk_perc,dif_l,dif_lk_perc,modelo,b1_id,b2_id,explanation,date_model_pred,date_self_pred
0,36.0,cualquiera podria ganar - aunque tiene mejor k...,0,0,0.475,"fail 19 % of 1150 (val, test)","WARNING: .45 to .65, fail 50 %",jesse rodriguez,sunny edwards,1.08 - 1.14 = -0.06,6005 - 7051 = -1046,2000 - 1996 = 4,163 - 160 = 3,18.0 - 20.0 = -2.0,0.61 - 0.2 = 0.41,0.0 - 0.0 = 0.0,0.0 - 0.0 = 0.0,SVC(probability=True),6502,6532,"{'len_text_boxer2 > -0.14': -0.2311, 'birth_b1...",2023-11-13 00:48:25.158735,2023-11-13 00:51:46.981118
1,64.0,1,0,1,0.935,"fail 19 % of 1150 (val, test)",NaN,yoanki urrutia,fiodor czerkaszyn,3.1 - 1.36 = 1.74,0 - 0 = 0,1996 - 1990 = 6,188 - 170 = 18,13.0 - 22.0 = -9.0,0.38 - 0.64 = -0.25,0.0 - 1.0 = -1.0,0.0 - 0.0 = 0.0,SVC(probability=True),6553,6363,"{'birth_b1 > 0.79': 0.1979, 'b1_wk <= -0.66': ...",2023-11-13 00:48:25.158735,2023-11-13 00:54:46.634854
2,65.0,1,0,1,0.935,"fail 19 % of 1150 (val, test)",NaN,eduardo nunez,oscar escandon,1.1 - 7.0 = -5.9,13683 - 2008 = 11675,1997 - 1980 = 17,168 - 173 = -5,24.0 - 27.0 = -3.0,1.0 - 0.67 = 0.33,1.0 - 6.0 = -5.0,0.0 - 0.67 = -0.67,SVC(probability=True),6554,2775,"{'birth_b1 > 0.79': 0.1987, 'len_text_boxer1 >...",2023-11-13 00:48:25.158735,2023-11-13 00:55:20.509433
3,68.0,1,0,1,0.912,"fail 19 % of 1150 (val, test)",NaN,joe cordina,edward vazquez,1.08 - 8.0 = -6.92,6351 - 0 = 6351,1992 - 1995 = -3,175 - 170 = 5,16.0 - 15.0 = 1.0,0.56 - 0.2 = 0.36,0.0 - 1.0 = -1.0,0.0 - 0.0 = 0.0,SVC(probability=True),6557,6574,"{'birth_b1 > 0.79': 0.2022, 'b1_wk <= -0.66': ...",2023-11-13 00:48:25.158735,2023-11-13 00:55:27.640765
4,69.0,1,0,1,0.928,"fail 19 % of 1150 (val, test)",NaN,robeisy ramirez,rafael espinoza,1.07 - 8.0 = -6.93,7087 - 0 = 7087,1994 - 1994 = 0,168 - 185 = -17,13.0 - 21.0 = -8.0,0.62 - 0.86 = -0.24,1.0 - 0.0 = 1.0,0.0 - 0.0 = 0.0,SVC(probability=True),6505,6575,"{'birth_b1 > 0.79': 0.1913, 'b1_wk <= -0.66': ...",2023-11-13 00:48:25.158735,2023-11-13 00:55:34.299382
5,70.0,1,0,1,0.940,"fail 19 % of 1150 (val, test)",NaN,emanuel navarrete,robson conceicao,1.1 - 7.0 = -5.9,12665 - 7632 = 5033,1994 - 1996 = -2,180 - 170 = 10,38.0 - 17.0 = 21.0,0.82 - 0.47 = 0.35,1.0 - 2.0 = -1.0,0.0 - 0.0 = 0.0,SVC(probability=True),4164,137,"{'len_text_boxer2 > -0.14': -0.2409, 'birth_b1...",2023-11-13 00:48:25.158735,2023-11-13 00:55:40.177645
6,71.0,1,0,1,0.926,"fail 19 % of 1150 (val, test)",NaN,shakur stevenson,edwin de los santos,1.08 - 8.0 = -6.92,9892 - 0 = 9892,1997 - 1999 = -2,175 - 173 = 2,20.0 - 16.0 = 4.0,0.5 - 0.88 = -0.38,0.0 - 1.0 = -1.0,0.0 - 0.0 = 0.0,SVC(probability=True),136,6576,"{'birth_b1 > 0.79': 0.2027, 'len_text_boxer1 >...",2023-11-13 00:48:25.158735,2023-11-13 00:55:52.497151
7,72.0,1,0,1,0.926,"fail 19 % of 1150 (val, test)",NaN,diego pacheco,marcelo esteban coceres,1.05 - 10.0 = -8.95,1466 - 0 = 1466,2001 - 1983 = 18,193 - 175 = 18,19.0 - 32.0 = -13.0,0.84 - 0.56 = 0.28,0.0 - 5.0 = -5.0,0.0 - 1.0 = -1.0,SVC(probability=True),6558,2219,"{'birth_b1 > 0.79': 0.1958, '-0.42 < len_text_...",2023-11-13 00:48:25.158735,2023-11-13 00:56:02.825364
8,73.0,1,3,1,0.710,"fail 10 % of 285 (warning: few data)(val, test)",NaN,david benavidez,demetrius andrade,1.3 - 3.5 = -2.2,12090 - 24084 = -11994,1996 - 1988 = 8,188 - 178 = 10,27.0 - 32.0 = -5.0,0.85 - 0.59 = 0.26,0.0 - 0.0 = 0.0,0.0 - 0.0 = 0.0,SVC(probability=True),95,1499,"{'len_text_boxer2 > -0.14': -0.2356, 'birth_b1...",2023-11-13 00:48:25.158735,2023-11-13 00:56:17.150294
9,75.0,1,0,1,0.935,"fail 19 % of 1150 (val, test)",NaN,michael conlan,jordan gill,1.17 - 5.5 = -4.33,9592 - 3207 = 6385,1992 - 1994 = -2,173 - 170 = 3,18.0 - 27.0 = -9.0,0.5 - 0.3 = 0.2,2.0 - 2.0 = 0.0,1.0 - 1.0 = 0.0,SVC(probability=True),6560,6577,"{'birth_b1 > 0.79': 0.1997, 'b1_wk <= -0.66': ...",2023-11-13 00:48:25.158735,2023-11-13 00:56:29.058517


In [ ]:
# logger.delete_row(64)

Proced to delete record
{'fight_id': 64.0, 'self_pred': 0, 'cluster': 0, 'boxer1_pred': 1, 'prob_win': 0.935, 'data_cuant': 'fail 19 % of 1150 (val, test)', 'confidence': nan, 'boxer1': 'yoanki urrutia', 'boxer2': 'fiodor czerkaszyn', 'dif_bet': '3.1 - 1.36 = 1.74', 'dif_len_text': '0 - 0 = 0', 'dif_edad': '1996 - 1990 = 6', 'dif_height': '188 - 170 = 18', 'dif_w': '13.0 - 22.0 = -9.0', 'dif_wk_perc': '0.38 - 0.64 = -0.25', 'dif_l': '0.0 - 1.0 = -1.0', 'dif_lk_perc': '0.0 - 0.0 = 0.0', 'modelo': 'SVC(probability=True)', 'b1_id': 6553, 'b2_id': 6363, 'explanation': "{'birth_b1 > 0.79': 0.1979, 'b1_wk <= -0.66': -0.1433, 'cluster <= 0.12': -0.0968, '-0.42 < len_text_boxer2 <= -0.38': 0.089, 'b1_more_fame <= -1.62': -0.0611, 'height_b1 > 0.22': 0.056, 'b1_momios_menores <= -1.02': -0.0559, '-0.81 < b1_lk <= -0.38': 0.0549, 'b2_invicto > 0.50': 0.054, 'len_text_boxer1 <= -0.48': -0.0534}"}
 (y/n): y
Saved data
Row deleted successfully



### export predictions

In [ ]:
export_preds(preds2)